# 課題J: 評価プロトコルの理解と実装

## 課題J-1 交差検証

1. ホールドアウト
       データの集まりをモデルを作る学習用データとモデルを評価するテストデータに分割する方法
       モデルを評価するテストデータは未知のデータである必要があるので、
       疑似的に未知のデータを作るこの手法が用いられる
       
       - ① データセットを学習データとテストデータに分割する
       - ② 学習データで数理モデルを作る
       - ③ テストデータでモデルを評価する
       
2. 交差検証
       用意したデータをいくつかに分割して、その分割したデータのうち一つをテストデータとして残りのデータを学習データとし、
       モデルを作成し評価する、次に前回テストデータには使用していないデータをテストデータに選択して残りのデータを学習データとしてモデルを作成し評価する。これを分割した回数繰り返し、その平均で評価する
       データの偏りが生まれにくいため正しい評価が出やすい
       
       - ① データセットをいくつかの数で分割する
       - ② 分割したデータの中から学習データとテストデータに分ける
       - ③ 学習データで数理モデルを作る
       - ④ テストデータでモデルを評価する
       - ⑤ これまでにテストデータとして選んでいないデータをテストデータに選び残りを学習データとして③に戻る
       - ⑥ すべてのデータをテストデータとして選び終えたら、それぞれのモデルの評価の平均をとる
       
      
3. ホールドアウトとの違い
        テストデータと学習データをホールドアウトは2分割, 交差検証は3分割以上で分ける方法
        交差検証では学習データを取り換えているので、データが偏る確率が少なくなる
        
3. メリット、デメリット

        メリット  : データが少ないときの評価に適している 
        デメリット: データ数が多すぎると時間がかかってしまう
        
        

In [1]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
train = pd.read_csv('./titanic/train.csv')
test =  pd.read_csv('./titanic/test.csv')


In [3]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

K = 5 
kf = KFold(n_splits=K, shuffle=True, random_state=0)

X = train.drop(columns=['Survived','Cabin','Name','PassengerId','Sex','Embarked','Ticket','Age'])
y = train['Survived']

score_train = 0
score_test = 0

for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    logistic_regression = LogisticRegression()
    logistic_regression.fit(X_train,y_train)
    
    pred_train = logistic_regression.predict(X_train)
    
    auccuracy = accuracy_score(pred_train, y_train)
    score_train += auccuracy
    
    pred_test = logistic_regression.predict(X_test)
    auccuracy = accuracy_score(pred_test, y_test)
    
    score_test += auccuracy

print(f'train_score : {score_train / K : .3f}')
print(f'test_score : {score_test / K : .3f}')

train_score :  0.686
test_score :  0.685


## 課題J-2 交差検証のバリエーション(StratifiedKFold)

  - 分布の比率を維持したまま学習用データとテスト用データに分けてくれる
  - 分布に大きな偏りがある場合はこちらを使用する


In [4]:
from sklearn.model_selection import StratifiedKFold

K = 5

skf = StratifiedKFold(n_splits=K, shuffle=True, random_state=0)

score_train = 0
score_test = 0


for train_index, test_index in skf.split(X,y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
#     logistic_regression.fit(X_train,y_train)
    clf.fit(X_train,y_train)
    
#     pred_train = logistic_regression.predict(X_train)
    pred_train = clf.predict(X_train)
    
    auccuracy = accuracy_score(pred_train, y_train)
    score_train += auccuracy
    
#     pred_test = logistic_regression.predict(X_test)
    pred_test = clf.predict(X_test)
    auccuracy = accuracy_score(pred_test, y_test)
    
    score_test += auccuracy

print(f'train_score : {score_train / K : .3f}')
print(f'test_score : {score_test / K : .3f}')

train_score :  0.687
test_score :  0.688


## 課題J-3 交差検証のバリエーション(GroupKFold)

  - 学習用データとテスト用データの間に同じグループが分割しないように分けてくれる
  - グループとは同じ人からデータをとっている場合など判別が容易となる組み合わせを作らないこと

In [5]:
group = train['Ticket']
count = 0
for rename in range(0,len(group.index)):
    group = group.replace(group.iloc[count], rename)
    count += 1

In [6]:
from sklearn.model_selection import GroupKFold

K = 5

gkf = GroupKFold(n_splits=K)

score_train = 0
score_test = 0


for train_index, test_index in gkf.split(X,y,group):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    logistic_regression = LogisticRegression()
    logistic_regression.fit(X_train,y_train)
    
    pred_train = logistic_regression.predict(X_train)
    
    auccuracy = accuracy_score(pred_train, y_train)
    score_train += auccuracy
    
    pred_test = logistic_regression.predict(X_test)
    auccuracy = accuracy_score(pred_test, y_test)
    
    score_test += auccuracy

print(f'train_score : {score_train / K : .3f}')
print(f'test_score : {score_test / K : .3f}')


train_score :  0.684
test_score :  0.685


終わり